<a href="https://colab.research.google.com/github/khal-drog0/Breast_Cancer_Histopathology_Classification/blob/main/Drive_After_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json /root/.kaggle

! chmod 600 /root/.kaggle/kaggle.json

! kaggle datasets download -d paultimothymooney/breast-histopathology-images

! unzip breast-histopathology-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1551_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1601_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1651_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1701_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1751_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1801_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1851_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1901_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y1951_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2001_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2251_class0.png  
  inflating: IDC_regular_ps50_idx5/9346/0/9346_idx5_x2351_y2301_class0.png  
  inflating: IDC_regular_

In [1]:
import numpy as np
import pandas as pd
import os
import itertools
import os, stat, time
from os.path import dirname as up
import shutil
import glob
from PIL import Image

In [2]:
from glob import glob
data = glob('../IDC_regular_ps50_idx5/**/*.png', recursive = True)

In [3]:
img_size = 32
num_channels = 3

In [4]:
import random

def randomSamples(imageData):
    lst = list()
    while(len(lst) != 15000):
        a = random.choice(imageData)

        if a not in lst:
            lst.append(a)

    return lst

In [5]:
import cv2
import matplotlib.pyplot as plt

images = list()
labels = list()
newData = list()

newData = randomSamples(data)

for i in newData:
    if i.endswith('.png'):
        label = i[-5]
        img = cv2.imread(i)
        img_1 = cv2.resize(img, (img_size, img_size))
        images.append(img_1)
        labels.append(label)

In [6]:
x = np.stack(images)

In [7]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(labels)

In [8]:
x = x / 255

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.2)

# Building VAE

Building the encoder

In [10]:
import tensorflow

In [11]:
a = tensorflow.keras.layers.Input(shape = (img_size, img_size, num_channels), name = 'encoder_input')

encoder_conv_layer1 = tensorflow.keras.layers.Conv2D(filters = 1, kernel_size = (3, 3), padding = 'same', strides = 1, name = 'encoder_conv_1')(a)
encoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name = 'encoder_norm_1')(encoder_conv_layer1)
encoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name = 'encoder_leakyrelu_1')(encoder_norm_layer1)

encoder_conv_layer2 = tensorflow.keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), padding = 'same', strides = 1, name = 'encoder_conv_2')(encoder_activ_layer1)
encoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_2")(encoder_conv_layer2)
encoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_2")(encoder_norm_layer2)

encoder_conv_layer3 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_3")(encoder_activ_layer2)
encoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_3")(encoder_conv_layer3)
encoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_3")(encoder_norm_layer3)

encoder_conv_layer4 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=2, name="encoder_conv_4")(encoder_activ_layer3)
encoder_norm_layer4 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_4")(encoder_conv_layer4)
encoder_activ_layer4 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_4")(encoder_norm_layer4)

encoder_conv_layer5 = tensorflow.keras.layers.Conv2D(filters=64, kernel_size=(3,3), padding="same", strides=1, name="encoder_conv_5")(encoder_activ_layer4)
encoder_norm_layer5 = tensorflow.keras.layers.BatchNormalization(name="encoder_norm_5")(encoder_conv_layer5)
encoder_activ_layer5 = tensorflow.keras.layers.LeakyReLU(name="encoder_activ_layer_5")(encoder_norm_layer5)

shape_before_flatten = tensorflow.keras.backend.int_shape(encoder_activ_layer5)[1:]
encoder_flatten = tensorflow.keras.layers.Flatten()(encoder_activ_layer5)

2022-05-07 11:08:52.214170: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-07 11:08:52.326841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/khal-drog0/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-05-07 11:08:52.326871: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-07 11:08:52.329019: I tensorflow/core/platform/cpu_feature_guard

In [12]:
latent_space_dim = 2

encoder_mu = tensorflow.keras.layers.Dense(units = latent_space_dim, name = 'encoder_mu')(encoder_flatten)
encoder_log_variance = tensorflow.keras.layers.Dense(units = latent_space_dim, name = 'encoder_log_variance')(encoder_flatten)

In [13]:
def sampling(mu_log_variance):
    mu, log_variance = mu_log_variance
    epsilon = tensorflow.keras.backend.random_normal(shape = tensorflow.keras.backend.shape(mu), mean = 0.0, stddev = 1.0)
    random_sample = mu + tensorflow.keras.backend.exp(log_variance / 2) * epsilon

    return random_sample

In [14]:
encoder_output = tensorflow.keras.layers.Lambda(sampling, name = 'encoder_output')([encoder_mu, encoder_log_variance])

In [15]:
encoder = tensorflow.keras.models.Model(a, encoder_output, name = 'encoder_model')

In [16]:
encoder.summary()

Model: "encoder_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 encoder_conv_1 (Conv2D)        (None, 32, 32, 1)    28          ['encoder_input[0][0]']          
                                                                                                  
 encoder_norm_1 (BatchNormaliza  (None, 32, 32, 1)   4           ['encoder_conv_1[0][0]']         
 tion)                                                                                            
                                                                                                  
 encoder_leakyrelu_1 (LeakyReLU  (None, 32, 32, 1)   0           ['encoder_norm_1[0][0

Building the decoder

In [17]:
decoder_input = tensorflow.keras.layers.Input(shape = (latent_space_dim), name = 'decoder_input')

decoder_dense_layer1 = tensorflow.keras.layers.Dense(units = np.prod(shape_before_flatten), name = 'decoder_dense_1')(decoder_input)

decoder_reshape = tensorflow.keras.layers.Reshape(target_shape = shape_before_flatten)(decoder_dense_layer1)

decoder_conv_tran_layer1 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_1")(decoder_reshape)
decoder_norm_layer1 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_1")(decoder_conv_tran_layer1)
decoder_activ_layer1 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_1")(decoder_norm_layer1)

decoder_conv_tran_layer2 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_2")(decoder_activ_layer1)
decoder_norm_layer2 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_2")(decoder_conv_tran_layer2)
decoder_activ_layer2 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_2")(decoder_norm_layer2)

decoder_conv_tran_layer3 = tensorflow.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), padding="same", strides=2, name="decoder_conv_tran_3")(decoder_activ_layer2)
decoder_norm_layer3 = tensorflow.keras.layers.BatchNormalization(name="decoder_norm_3")(decoder_conv_tran_layer3)
decoder_activ_layer3 = tensorflow.keras.layers.LeakyReLU(name="decoder_leakyrelu_3")(decoder_norm_layer3)

decoder_conv_tran_layer4 = tensorflow.keras.layers.Conv2DTranspose(filters=3, kernel_size=(3, 3), padding="same", strides=1, name="decoder_conv_tran_4")(decoder_activ_layer3)
decoder_output = tensorflow.keras.layers.LeakyReLU(name="decoder_output")(decoder_conv_tran_layer4 )

In [18]:
decoder = tensorflow.keras.models.Model(decoder_input, decoder_output, name = 'decoder_model')

In [19]:
decoder.summary()

Model: "decoder_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 2)]               0         
                                                                 
 decoder_dense_1 (Dense)     (None, 4096)              12288     
                                                                 
 reshape (Reshape)           (None, 8, 8, 64)          0         
                                                                 
 decoder_conv_tran_1 (Conv2D  (None, 8, 8, 64)         36928     
 Transpose)                                                      
                                                                 
 decoder_norm_1 (BatchNormal  (None, 8, 8, 64)         256       
 ization)                                                        
                                                                 
 decoder_leakyrelu_1 (LeakyR  (None, 8, 8, 64)       

Building the VAE

In [26]:
vae_input = tensorflow.keras.layers.Input(shape = (img_size, img_size, num_channels), name = 'VAE_input')

vae_encoder_output = encoder(vae_input)

vae_decoder_output = decoder(vae_encoder_output)

vae = tensorflow.keras.models.Model(vae_input, vae_decoder_output, name = 'VAE')

In [27]:
vae.summary()

Model: "VAE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 VAE_input (InputLayer)      [(None, 32, 32, 3)]       0         
                                                                 
 encoder_model (Functional)  (None, 2)                 109988    
                                                                 
 decoder_model (Functional)  (None, 32, 32, 3)         125571    
                                                                 
Total params: 235,559
Trainable params: 234,725
Non-trainable params: 834
_________________________________________________________________


In [28]:
def loss_func(encoder_mu, encoder_log_variance):

    def vae_reconstruction_loss(y_true, y_predict):
        reconstruction_loss_factor = 1000
        reconstruction_loss = tensorflow.keras.backend.mean(tensorflow.keras.backend.square(y_true - y_predict), axis = [1, 2, 3])

        return reconstruction_loss_factor * reconstruction_loss
    
    def vae_kl_loss(encoder_mu, encoder_log_variance):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance - tensorflow.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis = [1, 2, 3])
        return kl_loss

    def vae_kl_loss_metric(y_true, y_predict):
        kl_loss = -0.5 * tensorflow.keras.backend.sum(1.0 + encoder_log_variance - tensorflow.keras.backend.square(encoder_mu) - tensorflow.keras.backend.exp(encoder_log_variance), axis = [1, 2, 3])
        return kl_loss

    def vae_loss(y_true, y_predict):
        reconstruction_loss = vae_reconstruction_loss(y_true, y_predict)
        kl_loss = vae_kl_loss(y_true, y_predict)

        loss = reconstruction_loss + kl_loss
        return loss

    return vae_loss

In [29]:
vae.compile(optimizer = tensorflow.keras.optimizers.Adam(lr = 0.0005), loss = loss_func(encoder_mu, encoder_log_variance))

In [30]:
x_train = np.reshape(x_train, newshape=(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)) 
x_test = np.reshape(x_test, newshape=(x_test.shape[0], x_train.shape[1], x_train.shape[2], 3))

In [31]:
history = vae.fit(x_train, x_train, epochs = 25, batch_size = 64, shuffle = True, validation_data = (x_test, x_test))

Epoch 1/25
188/188 [==============================] - 75s 392ms/step - loss: 9659.5957 - val_loss: 2047.4183
Epoch 2/25
188/188 [==============================] - 62s 331ms/step - loss: 1780.9968 - val_loss: 1702.2659
Epoch 3/25
188/188 [==============================] - 61s 322ms/step - loss: 1617.4390 - val_loss: 1572.2428
Epoch 4/25
188/188 [==============================] - 60s 320ms/step - loss: 1516.1958 - val_loss: 1493.6730
Epoch 5/25
188/188 [==============================] - 59s 316ms/step - loss: 1450.6013 - val_loss: 1431.1204
Epoch 6/25
188/188 [==============================] - 63s 338ms/step - loss: 1406.6500 - val_loss: 1394.0505
Epoch 7/25
188/188 [==============================] - 60s 319ms/step - loss: 1369.1290 - val_loss: 1360.6627
Epoch 8/25
188/188 [==============================] - 59s 315ms/step - loss: 1341.3778 - val_loss: 1329.1656
Epoch 9/25
188/188 [==============================] - 59s 315ms/step - loss: 1317.7542 - val_loss: 1307.3596
Epoch 10/25
188/188

In [36]:
# Reduce learing rate if the learning seems to have become constant

rlronp = tensorflow.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    mode = 'min',
    min_lr = 10e-10,
    patience = 2,
    verbose = 1
    )

estop = tensorflow.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'min',
    patience = 3, 
    verbose = 1,
    restore_best_weights = True
)

history_with_callbacks = vae.fit(
    x_train, x_train,
    epochs = 25,
    validation_data = (x_test, x_test),
    callbacks = [rlronp, estop]
)

Epoch 1/25
375/375 [==============================] - 76s 201ms/step - loss: 1129.4749 - val_loss: 1127.6755 - lr: 5.0000e-04
Epoch 2/25
375/375 [==============================] - 70s 186ms/step - loss: 1129.3203 - val_loss: 1127.3115 - lr: 5.0000e-04
Epoch 3/25
375/375 [==============================] - 70s 188ms/step - loss: 1129.1930 - val_loss: 1127.0869 - lr: 5.0000e-04
Epoch 4/25
375/375 [==============================] - 69s 184ms/step - loss: 1129.1053 - val_loss: 1127.0281 - lr: 5.0000e-04
Epoch 5/25
375/375 [==============================] - 70s 188ms/step - loss: 1128.9830 - val_loss: 1126.9410 - lr: 5.0000e-04
Epoch 6/25
375/375 [==============================] - 68s 183ms/step - loss: 1129.0143 - val_loss: 1127.1938 - lr: 5.0000e-04
Epoch 7/25
375/375 [==============================] - ETA: 0s - loss: 1128.9780
Epoch 7: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
375/375 [==============================] - 69s 185ms/step - loss: 1128.9780 - val_loss:

In [38]:
# Reduce learing rate if the learning seems to have become constant

rlronp_2 = tensorflow.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    mode = 'min',
    min_lr = 1e-10,
    patience = 2,
    verbose = 1
    )

estop_2 = tensorflow.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'min',
    patience = 3, 
    verbose = 1,
    restore_best_weights = True
)

history_with_callbacks_2 = vae.fit(
    x_train, x_train,
    epochs = 25,
    validation_data = (x_test, x_test),
    callbacks = [rlronp_2, estop_2]
)

Epoch 1/25
375/375 [==============================] - 83s 220ms/step - loss: 1127.8441 - val_loss: 1126.0145 - lr: 2.0000e-05
Epoch 2/25
375/375 [==============================] - 85s 225ms/step - loss: 1127.8588 - val_loss: 1125.9727 - lr: 2.0000e-05
Epoch 3/25
375/375 [==============================] - 83s 221ms/step - loss: 1127.8499 - val_loss: 1126.0072 - lr: 2.0000e-05
Epoch 4/25
375/375 [==============================] - 84s 224ms/step - loss: 1127.8383 - val_loss: 1125.9456 - lr: 2.0000e-05
Epoch 5/25
375/375 [==============================] - 74s 196ms/step - loss: 1127.8374 - val_loss: 1125.9326 - lr: 2.0000e-05
Epoch 6/25
375/375 [==============================] - 69s 183ms/step - loss: 1127.8146 - val_loss: 1125.9417 - lr: 2.0000e-05
Epoch 7/25
375/375 [==============================] - 67s 180ms/step - loss: 1127.7698 - val_loss: 1125.8983 - lr: 2.0000e-05
Epoch 8/25
375/375 [==============================] - 68s 182ms/step - loss: 1127.8120 - val_loss: 1125.8972 - lr: 2.0

In [39]:
encoder.save("VAE_encoder.h5") 
decoder.save("VAE_decoder.h5") 
vae.save("VAE.h5")

In [41]:
# Loading the model

model = tensorflow.keras.models.load_model('EfficientNetB3_PBL.h5')

saved_encoder = tensorflow.keras.models.load_model('VAE_encoder.h5')
saved_decoder = tensorflow.keras.models.load_model('VAE_decoder.h5')
saved_vae = tensorflow.keras.models.load_model('VAE.h5')

In [40]:
# Testing the encoder and decoder

encoded_data = encoder.predict(x_test)
decoded_data = decoder.predict(encoded_data)

In [42]:
decoded_data.shape

(3000, 32, 32, 3)

In [53]:
encoded_train_data = encoder.predict(x_train)
decoded_train_data = decoder.predict(encoded_train_data)

encoded_test_data = encoder.predict(x_test)
decoded_test_data = decoder.predict(encoded_test_data)

vae_train_output = vae.predict(x_train)
vae_test_output = vae.predict(x_test)

In [54]:
print('decoded_train_data:', decoded_train_data.shape)
print('decoded_test_data:', decoded_test_data.shape)
print('vae_train_output:', vae_train_output.shape)
print('vae_test_output:', vae_test_output.shape)

decoded_train_data: (12000, 32, 32, 3)
decoded_test_data: (3000, 32, 32, 3)
vae_train_output: (12000, 32, 32, 3)
vae_test_output: (3000, 32, 32, 3)


In [55]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

m = [
    'acc', 
    tf.keras.metrics.Precision(), 
    tf.keras.metrics.Recall(), 
    tf.keras.metrics.AUC()
    ]

# opt = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad =  False)
opt = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')
model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = m)

/home/khal-drog0/.local/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [56]:
# Decoded Training and Testing data evaluation metrics

train_loss, train_accuracy, train_precision, train_recall, train_auc = model.evaluate(decoded_train_data, y_train)
test_loss, test_accuracy, test_precision, test_recall, test_auc = model.evaluate(decoded_test_data, y_test)



94/94 [==============================] - 16s 137ms/step - loss: 0.7438 - acc: 0.7237 - precision: 0.7237 - recall: 0.7237 - auc: 0.6460


In [57]:
# VAE Training and Testing data evaluation metrics

loss_train, accuracy_train, precision_train, recall_train, auc_train = model.evaluate(vae_train_output, y_train)
loss_test, accuracy_test, precision_test, recall_test, auc_test = model.evaluate(vae_test_output, y_test)

94/94 [==============================] - 11s 120ms/step - loss: 0.7440 - acc: 0.7237 - precision: 0.7237 - recall: 0.7237 - auc: 0.6457
